In [1]:
import re
import mafan
from mafan import text
from functools import reduce

In [2]:
infile = open("./conversions.txt", "r+")
s2t_dict = dict()

In [3]:
for line in infile:
    line = line.rstrip()
    arr = line.split()
    trad = arr[0]
    sim = arr[1]
    if sim not in s2t_dict:
        s2t_dict[sim] = [trad]
    else:
        s2t_dict[sim].append(trad)
s2t_dict['-'] = ['-']

In [4]:
def prepare(sentence):
    new = ""  # input to your tokenizer
    char_list = []  # punct / english to be omitted

    for char in list(sentence):
        if text.identify(char) is mafan.NEITHER:
            new += "-"  # sub - with non-chinese chars
            char_list.append(char)
            continue
        else:
            new += char

    return new, char_list

In [5]:
def sequencer(tokens, example):

    flags = [1] * len(example)
    sequence = []
    tokens.sort(key=lambda x: -len(x))
    if "." in tokens:
        dotCount = tokens.count(".")
        for d in range(dotCount):
            tokens.remove(".")
            tokens.append(".")

    for token in tokens:
        # print("Token " + token)
        for match in re.finditer(token, example):
            location = (token, match.span()[0], match.span()[1])
            valid = reduce(lambda x, y: x*y, flags[location[1]:location[2]])
            if valid:
                sequence.append(location)
                for i in range(location[1], location[2]):
                    flags[i] = 0
            else:
                continue
    sequence.sort(key=lambda x: x[1])
    result = [x[0] for x in sequence]
    return result

In [6]:
def tokenizer(sentence, original, n=8):
    text, charList = prepare(sentence)
    token_list = []
    input_text = text
    for k in range(n, 0, -1):
        candidates = [input_text[i:i + k]
                      for i in range(len(input_text) - k + 1)]

        for candidate in candidates:
            if '-' in candidate:
                continue
            elif candidate in s2t_dict:
                token_list.append(candidate)
                input_text = re.sub(candidate, '', input_text)
    token_set = list(set(token_list))

    sentence_parts = sentence.split()
    ordered_parts = sequencer(sentence_parts, original)

    string = " ".join(x for x in ordered_parts)

    final = sequencer(token_set+charList, string)

    return final

In [7]:
input = "香港大学为亚洲之国际大学，借国际化、创新性及跨范畴发挥其影响力；凭卓越研究、优秀教学、知识与技术之交流转移，吸引及培育全球英才；并透过参与环球事务、其地区影响力及与中国内地之紧密连系，为促进社会进步作出贡献。"
tokens = tokenizer(input, input)
output = " ".join(x for x in tokens)
print(output)

香港大学 为 亚洲 之 国际 大学 ， 借 国际化 、 创新 性 及 跨 范畴 发挥 其 影响力 ； 凭 卓越 研究 、 优秀 教学 、 知识 与 技术 之 流转 ， 吸引 及 培育 全球 英 才 ； 并 透过 参与 环球 事务 、 其 地区 影响力 及 与 国内 之 紧密 连系 ， 为 促进 社会 进步 作出 贡献 。


In [8]:
input = "一辆小轿车，一名女司机，竟造成9死24伤。日前，深圳市交警局对事故进行通报：从目前证据看，事故系司机超速行驶且操作不当导致。目前24名伤员已有6名治愈出院，其余正接受治疗，预计事故赔偿费或超一千万元。"
tokens = tokenizer(input, input)
output = " ".join(x for x in tokens)
print(output)

一 辆 小 轿车 ， 一 名 女 司机 ， 竟 造成 9 死 2 4 伤 。 日前 ， 深圳市 交警 局 对 事故 进行 通报 ： 从 目前 证据 看 ， 事故 系 司机 超速 行驶 且 操作 不当 导致 。 目前 2 4 名 伤员 已 有 6 名 治愈 出院 ， 其余 正 接受 治疗 ， 预计 事故 赔偿 费 或 超 一 千万 元 。


Jieba output   
一辆 小轿车 ， 一名 女司机 ， 竟 造成 9 死 24 伤 。 日前 ， 深圳市 交警 局对 事故 进行 通报 ： 从 目前 证据 看 ， 事故 系 司机 超速行驶 且 操作 不当 导致 。 目前 24 名 伤员 已有 6 名 治愈 出院 ， 其余 正 接受 治疗 ， 预计 事故 赔偿费 或超 一千万元 。